In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.framework import ops
import math
import glob
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
train_image_path = '../input/plant-pathology-2021-fgvc8/train_images'
test_image_path = '../input/plant-pathology-2021-fgvc8/test_images'
train_df_path = '../input/plant-pathology-2021-fgvc8/train.csv'
test_df_path = '../input/plant-pathology-2021-fgvc8/sample_submission.csv'

In [ ]:
df_train = pd.read_csv(train_df_path)
df_train.head()

In [ ]:
train_image_urls = [train_image_path+"/"+f for f in df_train['image']]
train_label_l = list(df_train.labels)
train_label_l = [s.split(" ") for s in train_label_l]
train_label =[]
for i in train_label_l:
    train_label.append(tuple(i))


In [ ]:
mlb = MultiLabelBinarizer() 
mlb.fit(train_label) 
print(mlb.classes_)
train_label_e = mlb.transform(train_label) 

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMG_SIZE = 256
def decode_img(img):
  # 압축된 문자열을 3D uint8 텐서로 변환합니다
  img = tf.image.decode_jpeg(img, channels=3)
  # `convert_image_dtype`은0~1 사이의 float 값으로 변환해줍니다.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # 이미지를 원하는 크기로 조정합니다.
  return tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

def process_path(file_path):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img

image_ds = tf.data.Dataset.list_files(train_image_urls)
image_ds = image_ds.map(process_path, num_parallel_calls=AUTOTUNE)

label_ds = tf.data.Dataset.from_tensor_slices(train_label_e)

train_ds = ds = tf.data.Dataset.zip((image_ds, label_ds))

In [ ]:
DATASET_SIZE = 18632
train_size = int(0.9 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)

train_ds = train_ds.take(train_size)
val_ds = train_ds.skip(train_size)

In [ ]:
batch_size = 32
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)


In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)


base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

In [ ]:
#with tf.device('/gpu:0'):
model = tf.keras.Sequential([
      base_model,
      layers.GlobalAveragePooling2D(),
      layers.Dense(6, activation='sigmoid')
    ])

model.compile(loss='binary_crossentropy', optimizer='adam')

model.summary()


history = model.fit(train_ds, epochs=10, validation_data = val_ds)